In [1]:
# Importação as bibliotecas necessárias para conexão e coleta dos dados
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import os

In [2]:
# Variaveis ambientes

host = os.environ.get('HOST_POSTGRE')
user = os.environ.get('USER_POSTGRE')
password = os.environ.get('PASSWORD_POSTGRE')

In [3]:
# Descrevo os parâmetros de conexão conforme configurado
db_server_info = {'host': host, 
                  'porta': 5432, 
                  'dbname': 'dbs_vacina',
                  'user': user, 
                  'password': password} 

conn_str = '{}://{}:{}@{}:{}/{}'.format('postgresql+psycopg2', 
                                  db_server_info['user'], 
                                  db_server_info['password'], 
                                  db_server_info['host'], 
                                  db_server_info['porta'], 
                                  db_server_info['dbname']) 

alchemyEngine = create_engine(conn_str, pool_recycle=3600)

postgreSQLConnection = alchemyEngine.connect() 

In [4]:
# Encontrando o id relacionado ao Distrito Federal

pd.read_sql_query("SELECT * FROM public.tbl_cidade WHERE uf = 'DF'", postgreSQLConnection)

,id,nm_municipio,uf,lat,lon,cd_ibge_reduzido
0,5300108,Brasília,DF,-15.8049,-47.88683,530010


In [5]:
# Dataframe de CEPs do DF

dfCepsDF = pd.read_sql_query("SELECT * FROM public.tbl_cep WHERE cidade_id = 5300108", postgreSQLConnection)
dfCepsDF.head()

,id,cep,cidade_id,lat,lon
0,18098,70002,5300108,-47.879545,-15.790642
1,18099,70003,5300108,-47.880634,-15.790649
2,18100,70037,5300108,-47.882790,-15.805816
3,18101,70040,5300108,-47.884265,-15.791226
4,18102,70041,5300108,-47.881478,-15.788280


In [6]:
# Dataframe de Estabelecimentos de Saúde do DF

ceps = dfCepsDF.id

dfEstab = pd.read_sql_query("SELECT * FROM public.tbl_estabelecimento", postgreSQLConnection)

dfEstabDF = dfEstab.loc[dfEstab['id_cep'].isin(ceps)].copy()
dfEstabDF.head()

,id,nm_razao_social,nm_fantasia,id_cep,id_natureza_juridica,id_tipo_estabelecimento,id_atividade_principal,lat,lon
300620,3937399,MARIA DO CARMO PINHEIRO,MARIA DO CARMO PINHEIRO,18125.0,4000.0,NaN,NaN,NaN,NaN
300686,3263398,PERIOCENTRO ODONTOLOGIA LTDA,PERIOCENTRO,18132.0,2000.0,NaN,NaN,NaN,NaN
300816,3025411,CENTRO CLINICO SANTO EXPEDITO LTDA,CLINICAR,18144.0,2000.0,NaN,NaN,NaN,NaN
301540,788740,AVALIAR DIGNOSTICO LTDA,AVALIAR DIAGNOSTICO,18150.0,2062.0,16.0,1.0,NaN,NaN
301612,5078962,DENTE CROSS LTDA,DENTE CROSS,18190.0,2000.0,NaN,NaN,NaN,NaN


In [7]:
# Quantidade de estabelecimentos de saúde no DF

dfEstabDF.shape

(8815, 9)

In [8]:
# Informações gerais do dataframe de estabelecimentos de saúde do DF

dfEstabDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8815 entries, 300620 to 312262
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       8815 non-null   int64  
 1   nm_razao_social          8815 non-null   object 
 2   nm_fantasia              8815 non-null   object 
 3   id_cep                   8815 non-null   float64
 4   id_natureza_juridica     8815 non-null   float64
 5   id_tipo_estabelecimento  4596 non-null   float64
 6   id_atividade_principal   4596 non-null   float64
 7   lat                      0 non-null      float64
 8   lon                      0 non-null      float64
dtypes: float64(6), int64(1), object(2)
memory usage: 688.7+ KB


In [9]:
# Dropando colunas com muitos valores ausentes

dfEstabDF.drop(columns = ['id_tipo_estabelecimento','id_atividade_principal','lat','lon'],axis=1,inplace=True)
dfEstabDF.head()

,id,nm_razao_social,nm_fantasia,id_cep,id_natureza_juridica
300620,3937399,MARIA DO CARMO PINHEIRO,MARIA DO CARMO PINHEIRO,18125.0,4000.0
300686,3263398,PERIOCENTRO ODONTOLOGIA LTDA,PERIOCENTRO,18132.0,2000.0
300816,3025411,CENTRO CLINICO SANTO EXPEDITO LTDA,CLINICAR,18144.0,2000.0
301540,788740,AVALIAR DIGNOSTICO LTDA,AVALIAR DIAGNOSTICO,18150.0,2062.0
301612,5078962,DENTE CROSS LTDA,DENTE CROSS,18190.0,2000.0


In [10]:
# Dataframe dos estabelecimentos de saúde que foram postos de vacinação no Brasil

dfEstabVacinacao = pd.read_sql_query("SELECT DISTINCT id_estabelecimento FROM public.tbl_vacinacao", postgreSQLConnection)
dfEstabVacinacao.rename(columns={"id_estabelecimento": "id"},inplace=True)
dfEstabVacinacao.head()

,id
0,2427265
1,2621673
2,2432528
3,2469197
4,7204159


In [11]:
# Dataframe dos postos de vacinação do DF

dfPostosDF = pd.merge(dfEstabDF,dfEstabVacinacao)
dfPostosDF.head()

,id,nm_razao_social,nm_fantasia,id_cep,id_natureza_juridica
0,181552,DEMARTAMENTO MEDICO CAMARA DOS DEPUTADOS,DEPARTAMENTO MEDICO CAMARA DOS DEPUTADOS,18146.0,1040.0
1,11150,UNIDADE BASICA DE SAUDE N 1 ASA SUL,UBS 01 ASA SUL,18150.0,1023.0
2,10537,HOSPITAL MATERNO INFANTIL DE BRASILIA HMIB,HOSPITAL MATERNO INFANTIL DE BRASILIA HMIB,18151.0,1023.0
3,10510,FUNDACAO UNIVERSIDADE DE BRASILIA,HOSPITAL UNIVERSITARIO DE BRASILIA,18193.0,1139.0
4,6362729,COMANDO DA MARINHA,HOSPITAL NAVAL DE BRASILIA,18193.0,1015.0


In [12]:
# Quantidade de postos de vacinação no DF

dfPostosDF.shape

(116, 5)

In [13]:
# Adicionando info de latitude e longitude nos postos de vacinação no DF

dfCepsDF.drop(columns=['cep','cidade_id'],inplace=True)
dfCepsDF.rename(columns={"id":"id_cep"},inplace=True)
dfPostosDF = pd.merge(dfPostosDF,dfCepsDF,on='id_cep')
dfPostosDF.head()

,id,nm_razao_social,nm_fantasia,id_cep,id_natureza_juridica,lat,lon
0,181552,DEMARTAMENTO MEDICO CAMARA DOS DEPUTADOS,DEPARTAMENTO MEDICO CAMARA DOS DEPUTADOS,18146.0,1040.0,-47.861282,-15.800669
1,11150,UNIDADE BASICA DE SAUDE N 1 ASA SUL,UBS 01 ASA SUL,18150.0,1023.0,-47.891575,-15.822105
2,10537,HOSPITAL MATERNO INFANTIL DE BRASILIA HMIB,HOSPITAL MATERNO INFANTIL DE BRASILIA HMIB,18151.0,1023.0,-47.897709,-15.823910
3,10510,FUNDACAO UNIVERSIDADE DE BRASILIA,HOSPITAL UNIVERSITARIO DE BRASILIA,18193.0,1139.0,-47.885490,-15.798626
4,6362729,COMANDO DA MARINHA,HOSPITAL NAVAL DE BRASILIA,18193.0,1015.0,-47.885490,-15.798626


In [14]:
# Adicionando informação de natureza jurídica dos postos de vacinação no DF

dfNatJur = pd.read_sql_query("SELECT * FROM public.tbl_natureza_juridica", postgreSQLConnection)
dfNatJur.rename(columns={"id":"id_natureza_juridica"},inplace=True)
dfPostosDF = pd.merge(dfPostosDF,dfNatJur,on='id_natureza_juridica')
dfPostosDF.head()

,id,nm_razao_social,nm_fantasia,id_cep,id_natureza_juridica,lat,lon,natureza_juridica
0,181552,DEMARTAMENTO MEDICO CAMARA DOS DEPUTADOS,DEPARTAMENTO MEDICO CAMARA DOS DEPUTADOS,18146.0,1040.0,-47.861282,-15.800669,ORGAO PUBLICO DO PODER LEGISLATIVO FEDERAL
1,11150,UNIDADE BASICA DE SAUDE N 1 ASA SUL,UBS 01 ASA SUL,18150.0,1023.0,-47.891575,-15.822105,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...
2,10537,HOSPITAL MATERNO INFANTIL DE BRASILIA HMIB,HOSPITAL MATERNO INFANTIL DE BRASILIA HMIB,18151.0,1023.0,-47.897709,-15.823910,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...
3,5117666,UNIDADE BASICA DE SAUDE N 5 ASA NORTE,UBS 5 GRANJA DO TORTO,18313.0,1023.0,-47.905893,-15.819210,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...
4,10758,UNIDADE BASICA DE SAUDE N 2 CRUZEIRO,UBS 2 CRUZEIRO,18314.0,1023.0,-47.944787,-15.789660,ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU D...


In [15]:
#Visão geral dos postos do DF

dfPostosDF.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116 entries, 0 to 115
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    116 non-null    int64  
 1   nm_razao_social       116 non-null    object 
 2   nm_fantasia           116 non-null    object 
 3   id_cep                116 non-null    float64
 4   id_natureza_juridica  116 non-null    float64
 5   lat                   116 non-null    float64
 6   lon                   116 non-null    float64
 7   natureza_juridica     116 non-null    object 
dtypes: float64(4), int64(1), object(3)
memory usage: 8.2+ KB


In [16]:
# Analisando os dados de natureza jurídica
dfPostosDF.natureza_juridica.value_counts()

ORGAO PUBLICO DO PODER EXECUTIVO ESTADUAL OU DO DISTRITO FEDERAL    107
ORGAO PUBLICO DO PODER EXECUTIVO FEDERAL                              4
SERVICO SOCIAL AUTONOMO                                               2
FUNDACAO ESTADUAL OU DO DISTRITO FEDERAL                              1
FUNDACAO FEDERAL                                                      1
ORGAO PUBLICO DO PODER LEGISLATIVO FEDERAL                            1
Name: natureza_juridica, dtype: int64

In [17]:
# Lista de todos os pontos de vacinação do DF encontrados na base de dados

pd.set_option("max_rows", None)
dfPostosDF.nm_fantasia

0         DEPARTAMENTO MEDICO CAMARA DOS DEPUTADOS
1                                   UBS 01 ASA SUL
2       HOSPITAL MATERNO INFANTIL DE BRASILIA HMIB
3                            UBS 5 GRANJA DO TORTO
4                                   UBS 2 CRUZEIRO
5                                   UBS 1 CRUZEIRO
6                                             HRAN
7                                  UBS 2 ASA NORTE
8                                  UBS 1 ASA NORTE
9                    UBS 3 ASA NORTE VILA PLANALTO
10                    HOSPITAL REGIONAL DO GUARA I
11                                    UBS 01 GUARA
12                        UBS 04 GUARA LUCIO COSTA
13                               UBS 01 ESTRUTURAL
14                                    UBS 02 GUARA
15                                    UBS 03 GUARA
16                                  UBS 01 SIA CPP
17                               UBS 02 ESTRUTURAL
18                                UBS 1 LAGO NORTE
19                             

In [18]:
# criando o mapa e o geodataframe
import folium as fl
import geopandas

m = fl.Map(location=[-15.7751257, -48.0785352],tiles='OpenStreetMap',zoom_control=False)

dfPostosDF.rename(columns={"lat":"long","lon":"lati"},inplace=True)

dfPostosGeo = geopandas.GeoDataFrame(dfPostosDF, 
geometry = geopandas.points_from_xy(dfPostosDF.long, dfPostosDF.lati))

In [19]:
#marcadores

from folium.plugins import MeasureControl

ubs = fl.FeatureGroup(name='UBS DF - Pontos de vacinação Covid-19')

for i in range(0,len(dfPostosGeo)):
  fl.Marker([dfPostosGeo.loc[i,'lati'],dfPostosGeo.loc[i,'long']],
  tooltip= dfPostosGeo.loc[i,'nm_fantasia'],
  icon=fl.Icon(color='blue')
  ).add_to(ubs)

 
m.add_child(ubs)
m.add_child(MeasureControl())
m.add_child(fl.LatLngPopup())
fl.LayerControl().add_to(m)
m

In [20]:
# Dataframe da tbl de vacinação no Brasil do primeiro periodo

dfVacinacao1p = pd.read_sql_query('SELECT * FROM "vwPrimeiroPeriodo"', postgreSQLConnection)
dfVacinacao1p.head()

,paciente_id,id_estabelecimento,data_aplicacao,dose_declarada
0,9082067,4024966,2021-03-25,1.0
1,9082041,9045023,2021-02-11,1.0
2,9082041,9045023,2021-03-11,2.0
3,9082042,2261413,2021-03-01,1.0
4,9082043,5709512,2021-03-28,1.0


In [21]:
dfVacinacao1p.shape

(26820139, 4)

In [22]:
# Dataframe da tbl de vacinação no Brasil no segundo periodo
dfVacinacao2p = pd.read_sql_query('SELECT * FROM "vwSegundoPeriodo"', postgreSQLConnection)
dfVacinacao2p.head()

,paciente_id,id_estabelecimento,data_aplicacao,dose_declarada
0,9082067,4024966,2021-04-26,2.0
1,9082043,17027,2021-04-19,2.0
2,9082045,7911076,2021-05-10,2.0
3,9082046,4049977,2021-05-12,1.0
4,9082051,2297213,2021-05-03,2.0


In [23]:
dfVacinacao2p.shape

(43558880, 4)

In [24]:
# Dataframe da tbl de vacinação no Brasil no terceiro periodo
dfVacinacao3p = pd.read_sql_query('SELECT * FROM "vwTerceiroPeriodo"', postgreSQLConnection)
dfVacinacao3p.head()

,paciente_id,id_estabelecimento,data_aplicacao,dose_declarada
0,9082064,2792591,2021-06-22,1.0
1,9082094,2023458,2021-07-09,1.0
2,9082096,2200988,2021-07-09,1.0
3,9082098,2269376,2021-07-13,1.0
4,9082100,11207,2021-06-27,NaN


In [25]:
dfVacinacao3p.shape

(44377024, 4)

In [26]:
dfVacinacao = pd.concat([dfVacinacao1p,dfVacinacao2p,dfVacinacao3p])
dfVacinacao.shape

(114756043, 4)

In [28]:
# Dataframe de vacinação do DF

postos = dfPostosDF.id
dfVacinacao = dfVacinacao.loc[dfVacinacao['id_estabelecimento'].isin(postos)]
dfVacinacao.head()

,paciente_id,id_estabelecimento,data_aplicacao,dose_declarada
112,9082347,9509356,2021-03-18,1.0
138,9082468,10790,2021-03-23,1.0
150,9082505,11150,2021-01-28,1.0
222,9082757,9509356,2021-03-18,1.0
530,9083681,11614,2021-02-26,1.0


In [29]:
#Verificando o tamanho do dataframe de vacinação do Df
dfVacinacao.shape

(1445836, 4)

In [ ]:
# Dataframe de pacientes no DF

dfPaciente = pd.read_sql_query('SELECT hash,id_cidade,id_cep,id_grupo FROM pos.tbl_paciente WHERE id_cidade = 5300108', postgreSQLConnection)
dfPaciente.head()

In [ ]:
# Merge das informações do cep da residência do paciente e do estabelecimento onde ele foi vacinado

dfVacinacao.rename(columns={"hash_paciente":"hash"},inplace=True)
dfMerged = pd.merge(dfVacinacao,dfPaciente,on='hash')
dfMerged.head()

In [ ]:
# Tamanho do Merge
dfMerged.shape

In [ ]:
# Checando as informações
dfMerged.info()

In [ ]:
# Verificando a quantidade de valores nulos de CEP
print(dfMerged.id_cep.isna().sum()/len(dfMerged))

In [ ]:
# Quase 30% dos vacinados do DF não possuem número do CEP da residência na base de dados. Não será possível fazer a análise da distância até o posto de vacinação desses pacientes. Iremos dropar do dataframe.

dfMerged = dfMerged.dropna()
dfMerged.shape

In [ ]:
# Retirada das pessoas que tomaram 2 doses para manter no dataset apenas pacientes unicos

dfMerged = dfMerged[dfMerged.dose_declarada == 1]
dfMerged.dose_declarada.value_counts()

In [ ]:
# Verificação do shape
dfMerged.shape

In [ ]:
# Agrupando os pacientes de acordo com o posto de vacinação e cep da residência
dfMerged_group = dfMerged[['estabelecimento_id','id_cep']].copy()
dfMerged_group = dfMerged_group.groupby(['estabelecimento_id','id_cep']).size().reset_index(name='contador')
dfMerged_group.head(10)

In [ ]:
# Valor máximo do contador (Pessoas com o mesmo cep que foram se vacinar no mesmo lugar)
dfMerged_group.contador.max()

In [ ]:
# Soma da coluna contador (deve ser igual o shape do dfMerged)
dfMerged_group.contador.sum()

In [ ]:
# Adição de latitude e longitude dos postos de vacinação
dfPostosDF.rename(columns={"id":"estabelecimento_id"},inplace=True)
dfMerged_group = pd.merge(dfMerged_group,dfPostosDF[['lat','lon','estabelecimento_id']],on='estabelecimento_id')
dfMerged_group.rename(columns={"lat":"lat_estab","lon":"lon_estab"},inplace=True)
dfMerged_group.head()

In [ ]:
# Adição de latitude e longitude das residências dos pacientes
dfMerged_group = pd.merge(dfMerged_group,dfCepsDF)
dfMerged_group.rename(columns={"lat":"lat_resid","lon":"lon_resid"},inplace=True)
dfMerged_group.head()

In [ ]:
# Checando informações
dfMerged_group.info()

In [ ]:
# Calculando a distancia entre as duas coordenadas

import haversine as hs
distancia = []
dfMerged_group['coor_estab'] = list(zip(dfMerged_group.lat_estab,dfMerged_group.lon_estab))
dfMerged_group['coor_resid'] = list(zip(dfMerged_group.lat_resid,dfMerged_group.lon_resid))

for index,row in dfMerged_group.iterrows():
    distancia.append(hs.haversine(row['coor_estab'],row['coor_resid']))

dfMerged_group['distancia'] = distancia
dfMerged_group.head()


In [ ]:
# Análise da variável distância

dfMerged_group.distancia.describe()

In [ ]:
# Histograma da variável distância

import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
sns.histplot(data=dfMerged_group,x='distancia')

In [ ]:
# Boxplot da variável distância

sns.boxplot(x=dfMerged_group["distancia"])

In [ ]:
# Investigando os ceps que são outliers no cálculo da distância

dfMerged_group[dfMerged_group.distancia > 60].id_cep.value_counts()

In [ ]:
# Cada CEP foi verificado no Google Maps e as coordenadas foram diferentes das informações do nosso dataset

ceps_outliers = [72410,72430,71503,70630,72594,72457,72415,73150,72549,73330]
cep_correcao = dfCepsDF.loc[dfCepsDF['id_cep'].isin(ceps_outliers)]

ind1 = cep_correcao.index
lat_alt = [-15.85177,-15.72028,-16.01446,-16.01721,-16.00695,-16.00918,-16.05279,-15.99645,-15.60197,-15.60946]
lon_alt = [-47.94817,-47.88104,-48.05283,-48.08201,-48.05993,-48.06418,-47.98158,-47.99895,-47.87104,-47.87104]

for i1,la, lo in zip(ind1,lat_alt, lon_alt):
  dfCepsDF.iloc[i1,1] = la
  dfCepsDF.iloc[i1,2] = lo

In [ ]:
# Corrigindo todo o dataframe e calculando as novas distâncias

dfMerged_group.drop(['lat_resid','lon_resid','coor_estab','coor_resid','distancia'],axis='columns',inplace=True)

dfMerged_group = pd.merge(dfMerged_group,dfCepsDF)
dfMerged_group.rename(columns={"lat":"lat_resid","lon":"lon_resid"},inplace=True)

dfMerged_group['coor_estab'] = list(zip(dfMerged_group.lat_estab,dfMerged_group.lon_estab))
dfMerged_group['coor_resid'] = list(zip(dfMerged_group.lat_resid,dfMerged_group.lon_resid))

distancia.clear()
for index,row in dfMerged_group.iterrows():
    distancia.append(hs.haversine(row['coor_estab'],row['coor_resid']))

dfMerged_group['distancia'] = distancia
dfMerged_group.head()


In [ ]:
# Análise da variável distância após correção

dfMerged_group.distancia.describe()

In [ ]:
# Boxplot da variável distância após alteração

sns.boxplot(x=dfMerged_group["distancia"])

In [ ]:
# Distâncias de forma decrescente
dfMerged_group.sort_values(by='distancia',ascending=False,inplace=True)
dfMerged_group.head()

In [ ]:
# Lista das residências de pacientes que percorreram mais de 30km para ir até o posto de vacinação

resid_maior30 = dfMerged_group[dfMerged_group.distancia > 30].id_cep.value_counts()
df_resid_maior30 = dfCepsDF.loc[dfCepsDF['id_cep'].isin(resid_maior30.index)]
df_resid_maior30.reset_index(inplace=True)
df_resid_maior30.head()

In [ ]:
# Análise dos pontos de vacinação mais cheios

dfUBS_count = dfMerged_group.groupby('estabelecimento_id')['contador'].agg('sum')
dfUBS_count = pd.merge(dfUBS_count,dfPostosDF[['estabelecimento_id','nm_fantasia','lat','lon']],on='estabelecimento_id')
dfUBS_count.sort_values(ascending=False,by='contador',inplace=True)
top10UBS = dfUBS_count.head(10)
top10UBS.reset_index(inplace=True,drop=True)
top10UBS


In [ ]:
# Criando marcadores

from folium.plugins import MeasureControl

m1 = fl.Map(location=[-15.7751257, -48.0785352],tiles='OpenStreetMap',zoom_control=False)

ubs = fl.FeatureGroup(name='UBS DF - Pontos de vacinação Covid-19')
for i in range(0,len(dfPostosGeo)): 
  fl.Marker([dfPostosGeo.loc[i,'lat'],dfPostosGeo.loc[i,'lon']],
  icon=fl.Icon(color='blue'),
  tooltip= dfPostosGeo.loc[i,'nm_fantasia'],
  ).add_to(ubs)
 
resid = fl.FeatureGroup(name='Deslocamento > 30 km')
for i in range(0,len(df_resid_maior30)):
  fl.Circle([df_resid_maior30.loc[i,'lat'],df_resid_maior30.loc[i,'lon']],
  color='red',
  ).add_to(resid)

top10 = fl.FeatureGroup(name='Pontos de vacinação mais cheios')
for i in range(0,len(top10UBS)):
  fl.Marker([top10UBS.loc[i,'lat'],top10UBS.loc[i,'lon']],
  icon=fl.Icon(color='red'),
  tooltip= top10UBS.loc[i,'nm_fantasia'],
  ).add_to(top10)

m1.add_child(resid)
m1.add_child(ubs)
m1.add_child(top10)
m1.add_child(MeasureControl())
m1.add_child(fl.LatLngPopup())
fl.LayerControl().add_to(m1)

m1.save('mapa.html')

m1

In [ ]:
#Construção do mapa de calor

dfHeat = dfMerged.groupby(['data_aplicacao','id_cep']).size().reset_index(name='contador')
dfHeat = pd.merge(dfHeat,dfCepsDF[['lat','lon','id_cep']],on='id_cep')
plotHeat = dfHeat[['lat','lon','contador']].values.tolist()

m2 = fl.Map(location=[-15.7751257, -48.0785352],tiles='OpenStreetMap',zoom_control=False)

from folium.plugins import HeatMap
heatmap = HeatMap(plotHeat,radius=18).add_to(m2)
heatmap.save('mapa_calor.html')
m2



In [ ]:
# Fechando conexão
postgreSQLConnection.close()